In [171]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [172]:
a = 6371229 # radius of the earth in [m]
def add_e1_e2(ds):
    # There is some averging going on here. Strictly the NEMO doc does not suggest doing this for
    # the scale factors but it seems to have been done in creating the AMM15 grid. It might becessary
    # to account for precision errors. Also the AMM15 claculation of e1u is a bit suspect because it
    # takes it as equal to e1f and e1v but it shold be equal to e1t
    part1 = np.square( ( np.deg2rad(ds.glamu[:,1:]).values - np.deg2rad(ds.glamu[:,:-1]).values ) * np.cos( np.deg2rad( ds.gphit[:,1:] ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphiu[:,1:]).values - np.deg2rad(ds.gphiu[:,:-1]).values ) )
    ds.e1t[:,1:] = a * np.sqrt( part1 + part2 ).mean("lon").broadcast_like(ds.e1t[:,1:]).values
    ds.e1t[:,0] = np.nan

    part1 = np.square( ( np.deg2rad(ds.glamv[1:,:]).values - np.deg2rad(ds.glamv[:-1,:]).values ) * np.cos( np.deg2rad( ds.gphit[1:,:] ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphiv[1:,:]).values - np.deg2rad(ds.gphiv[:-1,:]).values ) )
    ds.e2t[:] = ( a * np.sqrt( part1 + part2 ) ).mean()
    ds.e2t[0,:] = np.nan
       
    part1 = np.square( ( np.deg2rad(ds.glamv[:,1:]).values - np.deg2rad(ds.glamv[:,:-1]).values ) * np.cos( np.deg2rad( ds.gphif[:,:-1] ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphiv[:,1:]).values - np.deg2rad(ds.gphiv[:,:-1]).values ) )
    ds.e1f[:,:-1] = a * np.sqrt( part1 + part2 ).mean("lon").broadcast_like(ds.e1f[:,:-1]).values
    ds.e1f[:,-1] = np.nan

    part1 = np.square( ( np.deg2rad(ds.glamu[1:,:]).values - np.deg2rad(ds.glamu[:-1,:]).values ) * np.cos( np.deg2rad( ds.gphif[:-1,:] ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphiu[1:,:]).values - np.deg2rad(ds.gphiu[:-1,:]).values ) )
    ds.e2f[:] = ( a * np.sqrt( part1 + part2 ) ).mean().values
    ds.e2f[-1,:] = np.nan

    part1 = np.square( ( np.deg2rad(ds.glamt[:,1:]).values - np.deg2rad(ds.glamt[:,:-1]).values ) * np.cos( np.deg2rad( ds.gphiu[:,:-1] ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphit[:,1:]).values - np.deg2rad(ds.gphit[:,:-1]).values ) )
    ds.e1u[:,:-1] = a * np.sqrt( part1 + part2 ).mean("lon").broadcast_like(ds.e1u[:,:-1]).values
    ds.e1u[:,-1] = np.nan

    part1 = np.square( ( np.deg2rad(ds.glamf[1:,:]).values - np.deg2rad(ds.glamf[:-1,:]).values ) * np.cos( np.deg2rad( ds.gphiu[1:,:] ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphif[1:,:]).values - np.deg2rad(ds.gphif[:-1,:]).values ) )
    ds.e2u[:] = ( a * np.sqrt( part1 + part2 ) ).mean().values
    ds.e2u[0,:] = np.nan 

    part1 = np.square( ( np.deg2rad(ds.glamf[:,1:]).values - np.deg2rad(ds.glamf[:,:-1]).values ) * np.cos( np.deg2rad( ds.gphiv[:,1:]  ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphif[:,1:]).values - np.deg2rad(ds.gphif[:,:-1]).values ) )
    ds.e1v[:,1:] = a * np.sqrt( part1 + part2 ).mean("lon").broadcast_like(ds.e1v[:,1:]).values
    ds.e1v[:,0] = np.nan

    part1 = np.square( ( np.deg2rad(ds.glamt[1:,:]).values - np.deg2rad(ds.glamt[:-1,:]).values ) * np.cos( np.deg2rad( ds.gphiv[:-1,:]  ) ) )
    part2 = np.square( ( np.deg2rad(ds.gphit[1:,:]).values - np.deg2rad(ds.gphit[:-1,:]).values ) )
    ds.e2v[:] = ( a * np.sqrt( part1 + part2 ) ).mean().values
    ds.e2v[-1,:] = np.nan

In [173]:
# Load the AMM15 coordinates file
amm15coord_ds = xr.open_dataset( "/work/n01/n01/shared/CO_AMM15/P1_INPUTS/COORDINATES/amm15.coordinates.nc" )

# Coarsen the grid by taking every 5th grid point. This gives roughly 7.5 km resolution
# The northern and eastern most coordinates are the same in AMM15 and AMM75
lat_inds = np.arange(4,1345,5)
lon_inds = np.arange(2,1458,5)
amm75coord_ds = amm15coord_ds.isel(lat=lat_inds, lon=lon_inds)

In [174]:
# Add the horizontal scale factors e1 and e2
add_e1_e2(amm75coord_ds)

# Remove the nans around the edge which we couldn't calculate becuase we couldn't compute
# the derivate at the edge
lat_inds = np.arange(1,268,1)
lon_inds = np.arange(1,291,1)
amm75coord_ds = amm75coord_ds.isel(lat=lat_inds, lon=lon_inds).load()
amm75coord_ds.attrs["Description"] = "Anthoyn Wise, NOC, 26/03.24, created by coarsening the AMM15 coordinates using a stride of five to give approximately 7.5 km resolution"

In [170]:
# Save to disc
amm75coord_ds.to_netcdf( "/work/n01/n01/anwise/PROJECTS/PISCES/amm75_coordinates.nc" )